In [139]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout, LeakyReLU, BatchNormalization, Input, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import RandomNormal

In [140]:
def discriminatorModel(imageShape):
    model = Sequential()
    model.add(Conv2D(64, (4, 4), strides=(2,2), padding='same', input_shape=imageShape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (4, 4), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (4, 4), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (4, 4), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (4, 4), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Conv2D(1, (4,4)))
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    model.summary()

In [141]:
def residualBlock(n_filters, input_layer):
    init = RandomNormal(stddev=0.02)
    model = Sequential()
    model.add(Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Concatenate([model, input_layer],))
    return model

In [142]:
def generatorModel(LImageShape, LResnet=9):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=LImageShape)
    
    model = Sequential()
    model.add(Conv2D(64, (7,7), padding = 'same',  activation='relu', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), strides = (2,2), padding = 'same',  activation='relu', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Conv2D(258, (3,3), strides = (2,2), padding = 'same',  activation='relu', kernel_initializer=init))
    model.add(BatchNormalization())
    
    for _ in range(LResnet):
        model.add(residualBlock(256, model))
    
    model.add(Conv2DTranspose(128, (3,3), strides = (2,2), padding = 'same',  activation='relu', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(64, (3,3), strides = (2,2), padding = 'same',  activation='relu', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Conv2D(3, (7,7), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    
    #TO-DO: This part
    
    #out_image = Activation('tanh')(model)
    
    # define model
    #modelGenerator = Model(in_image, out_image)
    return model

In [143]:
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    g_model_1.trainable = True
    d_model.trainable = False
    g_model_2.trainable = False

    
    #TO-DO: ?
    
    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)

    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)

    output_f = g_model_2(gen1_out)
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)

    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer='adam')
    return model

In [144]:
def load_real_samples(filename):
    data = np.load(filename)
    X1, X2 = data['arr_0'], data['arr_1']
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

In [145]:
#TO-DO: Upload Images
image_shape = (256,256,3,1)
g_model_AtoB = generatorModel(image_shape)